In [1]:
import pandas as pd


def normalize_trade_date(value):
    """Normalize any incoming date to a single string format '%m/%d/%Y'."""
    if value is None:
        return None

    # Try pandas first (handles many formats)
    try:
        dt = pd.to_datetime(value, errors="coerce")
        if not pd.isna(dt):
            return dt.strftime("%m/%d/%Y")
    except Exception:
        pass

    from datetime import datetime
    s = str(value).strip()
    if not s:
        return None

    for fmt in ("%Y-%m-%d", "%d/%m/%Y", "%m/%d/%Y", "%m-%d-%Y"):
        try:
            dt = datetime.strptime(s, fmt)
            return dt.strftime("%m/%d/%Y")
        except ValueError:
            continue

    return None


def generate_trades(csv_file, portfolio_type='simple'):
    """
    Generate trade code from CSV.

    Args:
        csv_file: Path to CSV file. It must have:
            - one of: ticker, symbol, tvId, tv_Id
            - side, price, quantity columns
            - optional date/cts/mts column for trade date
        portfolio_type: 'simple' or 'full'
    """
    df = pd.read_csv(csv_file)

    # Build case-insensitive column lookup
    cols = {str(c).lower(): c for c in df.columns}

    def get_col(candidates, required=True, label=None):
        """Return the first matching column name from candidates (case-insensitive)."""
        label = label or candidates
        for cand in candidates:
            key = str(cand).lower()
            if key in cols:
                return cols[key]
        if required:
            raise ValueError(
                f"CSV must contain column(s) {candidates} for {label} (case-insensitive). "
                f"Found columns: {list(df.columns)}"
            )
        return None

    # Core columns
    ticker_col   = get_col(["ticker", "symbol", "tvid", "tv_id"], label="ticker")
    side_col     = get_col(["side"], label="side")
    price_col    = get_col(["price"], label="price")
    quantity_col = get_col(["quantity"], label="quantity")
    date_col     = get_col(["date", "cts", "mts"], required=False, label="date")

    # Optional parameters setup for full mode
    selected_params = set()
    asset_type_col = market_cap_col = industry_col = sector_col = None

    if portfolio_type != 'simple':
        print("Full portfolio mode: you can include optional parameters in generated add_trade calls.")
        print("You can choose each of: asset_type, market_cap, industry, sector.")

        ans = input("Include Asset Type? (Y/N): ").strip().lower()
        if ans in ("y", "yes"):
            selected_params.add("asset_type")

        ans = input("Include Market Cap? (Y/N): ").strip().lower()
        if ans in ("y", "yes"):
            selected_params.add("market_cap")

        ans = input("Include Industry? (Y/N): ").strip().lower()
        if ans in ("y", "yes"):
            selected_params.add("industry")

        ans = input("Include Sector? (Y/N): ").strip().lower()
        if ans in ("y", "yes"):
            selected_params.add("sector")

        # Try to map selected optional params to CSV columns if present
        if 'asset_type' in selected_params:
            asset_type_col = get_col(["asset_type"], required=False, label="asset_type")
        if 'market_cap' in selected_params:
            market_cap_col = get_col(["market_cap", "market cap"], required=False, label="market_cap")
        if 'industry' in selected_params:
            industry_col = get_col(["industry"], required=False, label="industry")
        if 'sector' in selected_params:
            sector_col = get_col(["sector"], required=False, label="sector")

    print("# Reset portfolio")
    print("reset_portfolio()\n")

    for _, row in df.iterrows():
        ticker = str(row[ticker_col]).lower()
        side   = str(row[side_col]).lower()
        price  = float(row[price_col])
        qty    = abs(float(row[quantity_col]))

        # Optional date column; normalize if present
        trade_date = normalize_trade_date(row[date_col]) if date_col is not None else None

        if portfolio_type == 'simple':
            if trade_date is not None:
                print(
                    f"add_trade(ticker='{ticker}', side='{side}', "
                    f"price={price}, quantity_buy={qty}, date='{trade_date}')"
                )
            else:
                print(
                    f"add_trade(ticker='{ticker}', side='{side}', "
                    f"price={price}, quantity_buy={qty})"
                )
        else:  # full
            # Build optional argument fragments based on user-selected params
            extra_parts = []

            if 'asset_type' in selected_params:
                if asset_type_col is not None and not pd.isna(row[asset_type_col]):
                    at_val = str(row[asset_type_col])
                    extra_parts.append(f"asset_type='{at_val}'")
                else:
                    extra_parts.append("asset_type=None")

            if 'market_cap' in selected_params:
                if market_cap_col is not None and not pd.isna(row[market_cap_col]):
                    mc_val = str(row[market_cap_col])
                    extra_parts.append(f"market_cap='{mc_val}'")
                else:
                    extra_parts.append("market_cap=None")

            if 'industry' in selected_params:
                if industry_col is not None and not pd.isna(row[industry_col]):
                    ind_val = str(row[industry_col])
                    extra_parts.append(f"industry='{ind_val}'")
                else:
                    extra_parts.append("industry=None")

            if 'sector' in selected_params:
                if sector_col is not None and not pd.isna(row[sector_col]):
                    sec_val = str(row[sector_col])
                    extra_parts.append(f"sector='{sec_val}'")
                else:
                    extra_parts.append("sector=None")

            # Always include ticker first, then optional parts, then side/price/qty/date
            base_parts = [f"ticker='{ticker}'"] + extra_parts + [
                f"side='{side}'",
                f"price={price}",
                f"quantity_buy={qty}",
            ]

            if trade_date is not None:
                base_parts.append(f"date='{trade_date}'")

            print("add_trade(" + ", ".join(base_parts) + ")")

    print("\n# Display")
    print("df = get_portfolio_df()")
    print("df")

In [2]:
# Example usage for portfolio_simple.ipynb
generate_trades('questdb-query-1764846052060.csv')

# Reset portfolio
reset_portfolio()

add_trade(ticker='eth_crypto', side='buy', price=2797.38187797189, quantity_buy=1.0, date='12/02/2025')
add_trade(ticker='xrp_crypto', side='buy', price=2.00578824078328, quantity_buy=3000.0, date='12/02/2025')
add_trade(ticker='aapl_nasdaq', side='buy', price=282.81, quantity_buy=2.0, date='12/02/2025')
add_trade(ticker='xrp_crypto', side='buy', price=2.04373488487825, quantity_buy=1.0, date='12/02/2025')
add_trade(ticker='aapl_nasdaq', side='buy', price=282.68, quantity_buy=20.0, date='12/02/2025')
add_trade(ticker='sol_crypto', side='buy', price=132.060646499455, quantity_buy=4.0, date='12/02/2025')
add_trade(ticker='doge_crypto', side='buy', price=0.1433955435275, quantity_buy=10000.0, date='12/02/2025')
add_trade(ticker='eth_crypto', side='hold', price=2997.78370130916, quantity_buy=1.0, date='12/02/2025')
add_trade(ticker='xrp_crypto', side='hold', price=2.15608012958207, quantity_buy=3000.0, date='12/02/2025')
add_trade(ticker='aapl_nasdaq', 